In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cv2
from keras.utils import image_dataset_from_directory
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf

2024-08-20 17:07:01.688520: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 17:07:01.688695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 17:07:01.962181: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
c=cv2.imread("/kaggle/input/skin-cancer-dataset/train_cancer/benign/14.jpg")
c.shape

(224, 224, 3)

In [4]:
df="/kaggle/input/skin-cancer-dataset/train_cancer"

In [5]:
# for img,label in training_data.take(1):
#     print(img.shape,label.shape)
#     print(np.unique(label))

In [6]:
training_data=image_dataset_from_directory(
df,
seed=123,
labels='inferred',
label_mode='categorical',
validation_split=0.2,
subset="training",
batch_size=32,
image_size=(224,224))

validation_data=image_dataset_from_directory(
df,
labels='inferred',
label_mode='categorical',
seed=123,
validation_split=0.2,
subset="validation",
batch_size=32,
image_size=(224,224))

Found 270 files belonging to 2 classes.
Using 216 files for training.
Found 270 files belonging to 2 classes.
Using 54 files for validation.


In [7]:
for images, labels in training_data:
    print("Label batch shape:", labels.shape)
    print("Unique labels:", np.unique(labels))
    break

Label batch shape: (32, 2)
Unique labels: [0. 1.]


# # Deep Learning

In [8]:
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,Input
from keras.models import Model
from keras.regularizers import l2
from keras.initializers import HeNormal
from keras.callbacks import EarlyStopping

In [9]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [10]:
inputs=Input(shape=(224,224,3))
c=Conv2D(32,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
c=Conv2D(32,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)

c=Conv2D(64,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(b)
c=Conv2D(64,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)

c=Conv2D(32,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(b)
c=Conv2D(32,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)

c=Conv2D(128,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(b)
c=Conv2D(128,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)

c=Conv2D(32,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(b)
c=Conv2D(32,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)

c=Conv2D(256,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(b)
c=Conv2D(256,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)
f=Flatten()(b)
d=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(f)
d=Dense(2,activation="sigmoid")(d)
model=Model(inputs,d)
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=["accuracy"])

In [11]:
model.fit(training_data,epochs=50,validation_data=validation_data,callbacks=[early_stopping])

Epoch 1/50
1/7 ━━━━━━━━━━━━━━━━━━━━ 3:29 35s/step - accuracy: 0.5625 - loss: 23.2600

I0000 00:00:1724173676.440323     116 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1724173676.472174     116 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7789 - loss: 22.7753   

W0000 00:00:1724173699.204156     115 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1724173700.876259     115 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


7/7 ━━━━━━━━━━━━━━━━━━━━ 67s 5s/step - accuracy: 0.7886 - loss: 22.7365 - val_accuracy: 0.9630 - val_loss: 24.5358
Epoch 2/50
1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.8750 - loss: 21.7627

W0000 00:00:1724173708.383224     114 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.8870 - loss: 21.4821 - val_accuracy: 0.9630 - val_loss: 23.7052
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.9322 - loss: 20.1576 - val_accuracy: 0.9630 - val_loss: 23.0156
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.9559 - loss: 18.7247 - val_accuracy: 0.9630 - val_loss: 19.4011
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.9448 - loss: 17.3876 - val_accuracy: 0.9630 - val_loss: 17.0957
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.9915 - loss: 16.0428 - val_accuracy: 0.9630 - val_loss: 15.8401
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.9970 - loss: 14.7854 - val_accuracy: 0.9630 - val_loss: 14.1580
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.9758 - loss: 13.6540 - val_accuracy: 0.9630 - val_loss: 13.0227
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.9773 - loss: 12.6152 - val_accuracy: 0.7593 - val_l